In [59]:
import torch
import pickle
import random

from sklearn.decomposition import PCA



class close_words:
    def __init__(self,model,w2i,pca_1=False,n=5):
        self.model=model
        self.w2i=w2i
        self.n=n
        self.i2w=dict(zip(w2i.values(),w2i.keys()))
        pca = PCA(n_components=10)
        self.embeddings=model if not pca_1 else torch.tensor(pca.fit_transform(model))
 
    def input_word(self,word):
        self.sample_idx=self.w2i[word]
        self.sample_vec=self.embeddings[self.sample_idx]
    
    def sample_word(self):
        
        self.sample_idx=random.sample(list(self.w2i.values()),1)[0]
        self.sampled_word=self.i2w[self.sample_idx]
        self.sample_vec=self.embeddings[self.sample_idx]
        print('임의의 단어:',self.sampled_word)
    
    def l2_dist(self):
        
        trial=torch.mul((self.embeddings-self.sample_vec),(self.embeddings-self.sample_vec))
        trial[self.sample_idx]=100    
        trial=torch.sum(trial,dim=1)
        #rec=torch.argmax(-self.trial).item()
        idexes=torch.argsort(-trial, descending=True)[:self.n]
        step=1
        print('사전사이즈:',len(self.i2w))
        for idx in idexes:
            if idx==len(w2i): #unk 제거
                continue
            print(step,'번째 가까운 단어:',self.i2w[idx.item()])
            step+=1
                
    def cos_sim(self):
        
   
        upper=torch.sum(torch.mul(self.embeddings,self.sample_vec),dim=1)
  
        lower=torch.sum(torch.mul(self.embeddings,self.embeddings),dim=1)+torch.sum(torch.mul(self.sample_vec,self.sample_vec))
    
        trial=torch.div(upper,lower)

        trial[self.sample_idx]=-100
        idexes=torch.argsort(trial, descending=True)
        step=1
        for idx in idexes:
            if idx==len(w2i)+1: #unk 제거
                continue
            if '1학년' in self.i2w[idx.item()][1] or '실험' in self.i2w[idx.item()][1]:
                continue
            print(step,'번째 가까운 단어:',self.i2w[idx.item()])
            step+=1
            if step==self.n:
                break
        

In [58]:
if '1학년' in '안녕 실험' or '실험' in '실험':
    print('hi')

hi


## Word2Vec

In [36]:
#model=torch.load('./w2v_withoud_ns',map_location='cpu')
model=torch.load('./training_with_mecab_2018_2_epoch_14',map_location='cpu')['state_dict']['embedding_in.weight'].detach()

with open('./preprocessing//vocab_mecab.pickle','rb') as f:
    w2i=pickle.load(f)
    
print('임베딩 사이즈:',model.shape)  #unk 랑 embedding이 0부터인것을 몰랐음. 0은 버리면 됨
print('사전 사이즈:',len(w2i))  

FileNotFoundError: [Errno 2] No such file or directory: './training_with_mecab_2018_2_epoch_14'

In [256]:
showing=close_words(model,w2i,pca_1=True,n=5)
showing.sample_word()
showing.l2_dist()
print('임의의 단어',showing.sampled_word)
showing.cos_sim()

임의의 단어: ('제목', 'NNG')
사전사이즈: 30654
1 번째 가까운 단어: ('자본', 'NNG')
2 번째 가까운 단어: ('문사', 'NNG')
3 번째 가까운 단어: ('맛보기', 'NNG')
4 번째 가까운 단어: ('철두철미', 'MAG')
5 번째 가까운 단어: ('니까요', 'VCP+EC')
임의의 단어 ('제목', 'NNG')
1 번째 가까운 단어: ('맛보기', 'NNG')
2 번째 가까운 단어: ('문사', 'NNG')
3 번째 가까운 단어: ('자본', 'NNG')
4 번째 가까운 단어: ('영양소', 'NNG')
5 번째 가까운 단어: ('익힌', 'VV+ETM')


In [268]:
showing.input_word(('과제', 'NNG'))
showing.l2_dist()
print('====')
showing.cos_sim()

사전사이즈: 30654
1 번째 가까운 단어: ('있', 'VA')
2 번째 가까운 단어: ('없', 'VA')
3 번째 가까운 단어: ('강의', 'NNG')
4 번째 가까운 단어: ('문제', 'NNG')
5 번째 가까운 단어: ('같', 'VA')
====
1 번째 가까운 단어: ('있', 'VA')
2 번째 가까운 단어: ('없', 'VA')
3 번째 가까운 단어: ('강의', 'NNG')
4 번째 가까운 단어: ('않', 'VX')
5 번째 가까운 단어: ('같', 'VA')


## Doc2Vec

In [84]:
#model=torch.load('./w2v_withoud_ns',map_location='cpu')
model=torch.load('./doc2vec_2017_2_epoch_4',map_location='cpu')['state_dict']['lecture.weight'].detach()

with open('./preprocessing//doc_id_2017_2.pickle','rb') as f:
    w2i=pickle.load(f)
    
print('임베딩 사이즈:',model.shape)  #unk 랑 embedding이 0부터인것을 몰랐음. 0은 버리면 됨
print('사전 사이즈:',len(w2i))  

임베딩 사이즈: torch.Size([2827, 256])
사전 사이즈: 2827


In [93]:
showing=close_words(model,w2i,pca_1=True,n=5)
showing.sample_word()
showing.l2_dist()
print('임의의 단어',showing.sampled_word)
showing.cos_sim()

임의의 단어: ('김채연 교수님', '인지신경과학(영강)')
사전사이즈: 2827
1 번째 가까운 단어: ('이경문 교수님', '과목26')
2 번째 가까운 단어: ('윤진 교수님', '프랑스어텍스트독해')
3 번째 가까운 단어: ('Peter L. Sinclair 교수님', 'ACADEMIC ENGLISH ∥(영강) 심리1, 31-끝')
4 번째 가까운 단어: ('Fang-Chi Lu 교수님', '마케팅원론(영강)')
5 번째 가까운 단어: ('김준연 교수님', '중국고전강독II(외국어강의)')
임의의 단어 ('김채연 교수님', '인지신경과학(영강)')
1 번째 가까운 단어: ('윤진 교수님', '프랑스어텍스트독해')
2 번째 가까운 단어: ('Fang-Chi Lu 교수님', '마케팅원론(영강)')
3 번째 가까운 단어: ('이경문 교수님', '과목26')
4 번째 가까운 단어: ('Peter L. Sinclair 교수님', 'ACADEMIC ENGLISH ∥(영강) 심리1, 31-끝')


In [95]:
showing.input_word(('석흥일 교수님', '뇌및의공학입문(영강)'))
showing.l2_dist()
print('====')
showing.cos_sim()

사전사이즈: 2827
1 번째 가까운 단어: ('이창규 교수님', '진단검사의학임상실습 ※의과대학 의학과 전공과목(타 과 학생 수강신청 불가)')
2 번째 가까운 단어: ('이기성 교수님', '의학영상처리 바이오의공학부 / 생체의공학과')
3 번째 가까운 단어: ('김자영,양웅 교수님', '건축설계Ⅷ(캡스톤디자인)(영강) 제2공학관 536')
4 번째 가까운 단어: ('손상영 교수님', '컴퓨터언어및실습(영강) (건축사회환경공학부)')
5 번째 가까운 단어: ('윤진 교수님', '프랑스어텍스트독해')
====
1 번째 가까운 단어: ('이창규 교수님', '진단검사의학임상실습 ※의과대학 의학과 전공과목(타 과 학생 수강신청 불가)')
2 번째 가까운 단어: ('김자영,양웅 교수님', '건축설계Ⅷ(캡스톤디자인)(영강) 제2공학관 536')
3 번째 가까운 단어: ('윤진 교수님', '프랑스어텍스트독해')
4 번째 가까운 단어: ('이기성 교수님', '의학영상처리 바이오의공학부 / 생체의공학과')


In [96]:
import pandas as pd
data=pd.DataFrame(model.numpy())
data.index=list(w2i.keys())
data.to_csv('./2017_2_doc2vec.csv')

In [98]:
pd.read_csv('./2017_2_doc2vec.csv',index_col=0)

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
"('추진아 교수님', '건강과식이(영강)')",-0.053845,-0.568948,1.244892,-0.786648,-0.840090,-0.501724,0.328730,0.313001,0.411073,-0.132201,...,-0.268042,1.173327,-1.134598,-1.111604,-1.431616,0.108275,0.552451,-0.170885,-0.016268,-0.187759
"('전진영 교수님', '간호진단 11:00-11:50')",0.249143,-1.503443,-0.929815,0.604694,0.342592,0.378118,-0.219079,0.366403,2.017240,-0.651856,...,-0.308877,-0.894335,-0.023579,0.757573,-0.815016,1.059195,-0.256289,0.722908,-0.641910,0.158531
"('설근희 교수님', '기초간호과학I')",0.077894,-0.725161,-0.394855,0.229766,0.349132,-0.840524,-0.331633,-0.678769,0.042987,-0.819089,...,1.226520,0.942899,-0.530307,0.290129,-0.546145,0.769991,0.219948,1.290979,-0.033573,-0.977804
"('설근희,강푸름 교수님', '기초간호과학IV')",2.026756,0.910633,-1.339749,2.135214,-1.139028,-0.196394,0.494996,0.418037,-0.280714,0.534799,...,0.577609,-1.229095,-0.760696,-1.809626,-0.726953,-1.276671,0.572831,-0.424770,-1.103836,-0.986298
"('설근희,강푸름 교수님', '기초간호과학실습I')",-0.044091,0.453828,0.958725,-0.955989,0.216332,0.951665,-2.125130,-0.490218,-1.037207,-1.947201,...,-0.672551,0.629240,0.295675,-0.043486,-0.578873,-0.767685,-1.043650,-0.994682,0.539541,-1.517018
"('전진영 교수님', '생명윤리')",-1.115263,1.166594,-0.553729,0.181511,-0.144142,0.072894,0.410463,-1.553197,-0.959783,-0.714720,...,-0.081777,1.145848,0.696332,0.635177,-0.500391,-0.021937,-0.131771,1.252406,0.445635,0.117355
"('장성옥 교수님', '기본간호학II')",-0.186846,-0.518182,1.043228,-1.585833,-1.906533,0.508303,1.069136,-1.528181,-0.399128,-0.822482,...,-2.130933,-1.950293,-0.789132,0.180999,-0.836531,0.569506,-0.506628,1.722025,0.011521,-0.475698
"('장성옥 교수님', '기본간호학실습II')",0.513472,-1.454583,-0.340444,-1.225397,-0.586036,0.272552,-0.021199,-0.676650,0.176275,-0.030293,...,-0.305660,1.620820,0.885914,0.448986,-0.281396,-0.723747,1.150927,0.909734,0.150471,-4.089146
"('박원희 교수님', '기본간호학실습II')",0.401776,-0.346110,-1.017534,-0.453455,0.971597,0.382824,-1.172917,0.638582,0.170267,0.102960,...,-0.844921,-0.376434,-0.349196,0.737826,-0.828023,0.912171,0.321867,1.302093,-0.219511,0.798300
"('박원희 교수님', '기본간호학실습II 14:30-15:45')",-0.049199,-0.904124,1.356762,-1.327932,-1.229584,-0.549353,-0.618841,0.317145,-0.085366,-0.843500,...,-0.161826,1.223495,1.538576,2.261762,1.130011,0.508344,0.870894,0.382476,0.366949,-1.563003


## UNK 빈도수

In [98]:
with open('./preprocessing/vocab_without_josa_gut_su_freq.pickle','rb') as f:
    total_freq=pickle.load(f)
    
with open('./preprocessing/vocab_without_josa_gut_su.pickle','rb') as h:
    voc=pickle.load(h)
    
not_unk=list(voc.keys())

freq_sum=0

def summing(x):
    
    global freq_sum
    freq_sum+=x

_=list(map(lambda x:summing(total_freq[x]),not_unk))

ratio=(sum(total_freq.values())-freq_sum)/sum(total_freq.values())
print(ratio)

0.014246669941204219


In [102]:
## unk 빈도수가 문제는 아닌듯?? 

In [7]:
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: FEDA-124D

 C:\tensor_code\kluebot\Recommendation_System\embedding_re 디렉터리

2019-12-24  오후 12:20    <DIR>          .
2019-12-24  오후 12:20    <DIR>          ..
2019-12-23  오후 07:18    <DIR>          .ipynb_checkpoints
2019-12-22  오후 10:16             1,601 doc2vec_model.py
2019-12-22  오후 03:13        54,120,715 doc2vec_training_set_2017_1.pickle
2019-12-23  오후 07:18    <DIR>          preprocessing
2019-12-24  오후 12:20             7,241 sampling_sim.ipynb
2019-12-21  오후 05:07    <DIR>          skipgram_dataset
2019-12-23  오후 07:18             4,834 train_doc.py
2019-12-22  오후 03:22             5,821 train_ns.py
2019-12-18  오전 11:43             2,399 w2v_ns.py
2019-12-23  오전 12:28    <DIR>          __pycache__
               6개 파일          54,142,611 바이트
               6개 디렉터리   9,386,196,992 바이트 남음


In [8]:
import pandas as pd
pd.read_csv('../recommendation/tfidf/lecture_vector_2017_1.csv').tail()

,0,1,2,3,4,5,6,7,8,9,...,189,190,191,192,193,194,195,196,197,198
2684,"('박성근 교수님', '물리교과교육론')","('그냥', 'MAG') ('아무것', 'NNG') ('안', 'MAG') ('했'...",0.213964,0.0,0.000000,0.0,0.000000,0.220355,0.133806,0.000000,...,0.097532,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2685,"('김지혜 교수님', '화학교과교육론')","('초반', 'NNG') ('교수', 'NNG') ('수업', 'NNG') ('진행...",0.000000,0.0,0.000000,0.0,0.000000,0.097078,0.000000,0.000000,...,0.171871,0.0,0.072503,0.0,0.0,0.0,0.0,0.0,0.095118,0.0
2686,"('한신 교수님', '지구과학교과교육론')","('학습', 'NNG') ('량', 'NNG') ('보통', 'NNG') ('이'...",0.000000,0.0,0.000000,0.0,0.157612,0.000000,0.000000,0.000000,...,0.106368,0.0,0.179482,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2687,"('성국현 교수님', '군사학Ⅰ')","('고려', 'NNG') ('대학교', 'NNG') ('학우', 'NNG') ('라...",0.000000,0.0,0.712342,0.0,0.000000,0.000000,0.000000,0.276746,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2688,"('이보선 교수님', '군사학Ⅲ')","('참으로', 'MAG') ('의미', 'NNG') ('깊', 'VA') ('수업'...",0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
